In [1]:
import pandas as pd
import numpy as np
import tensorflow
import cv2 
import os

In [2]:
TRAIN_DIR = '/datasets/louis/train'
IMG_SIZE = 224

In [3]:
import sys
sys.path.append('../..')
from invasive_species.src.process_data import process_data

In [4]:
x_train, img_nums = process_data(TRAIN_DIR, 50, IMG_SIZE)

In [5]:
labels = pd.read_csv('../data/train_labels.csv')

In [6]:
len(x_train)

50

In [7]:
y_train = []
for n in img_nums:
    label = labels[labels['name']==int(n)]['invasive']
    y_train.append(label.iloc[0])

In [8]:
len(y_train)

50

# sckit Learn

In [9]:
from sklearn.neural_network import MLPClassifier

In [10]:
clf = MLPClassifier(solver='adam', max_iter=5, random_state=0)

In [11]:
x_train_np = np.array(x_train)

In [12]:
nb, x, y = x_train_np.shape
x_train_2d = x_train_np.reshape(nb, x*y)

In [13]:
clf.fit(x_train_2d, y_train)

/opt/venv/lib/python3.7/site-packages/sklearn/neural_network/_multilayer_perceptron.py:585: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (5) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)


MLPClassifier(max_iter=5, random_state=0)

In [14]:
import pickle
filename = 'model_sklearn_mlpclassifier.pkl'
pickle.dump(clf, open('../data/'+filename, 'wb'))

# AutoKeras

In [15]:
import autokeras as ak

In [16]:
clf = ak.ImageClassifier(max_trials=1)

In [17]:
clf.fit(np.array(x_train), np.array(y_train), epochs=1)

Trial 1 Complete [00h 00m 08s]
val_loss: 4.740097522735596

Best val_loss So Far: 4.740097522735596
Total elapsed time: 00h 00m 08s
INFO:tensorflow:Oracle triggered exit
2/2 [==============================] - 11s 4s/step - loss: 0.9494 - accuracy: 0.4183
INFO:tensorflow:Assets written to: ./image_classifier/best_model/assets


In [18]:
model = clf.export_model()

In [19]:
try:
    model.save("model_autokeras", save_format="tf")
except:
    model.save("model_autokeras.h5")

INFO:tensorflow:Assets written to: model_autokeras/assets


In [20]:
#y_test_pre = clf.predict(x_test)